In [1]:
# 01_CAPTURE_DRIVE.ipynb
# %% [markdown]
"""
# 01_CAPTURE_DRIVE: Captura de "Projeto MBA" e Upload ao MinIO

Sincroniza todo o conteúdo da pasta "PROJETO MBA" (remote `gdrive:` configurado com root_folder_id)
para um diretório temporário, e envia cada arquivo ao bucket `raw-data/projeto-mba/` no MinIO.
"""
# %% [code]
import os
import tempfile
import subprocess
from minio import Minio

# %% [code]
# 1) Preparar diretório temporário
tmp_dir = tempfile.mkdtemp(prefix="projeto_mba_")
print(f"Diretório temporário criado: {tmp_dir}")

# %% [code]
# 2) Sincronizar do remote gdrive: para local
e = subprocess.run([
    'rclone', 'copy', 'gdrive:', tmp_dir,
    '--create-empty-src-dirs'
], capture_output=True, text=True)
if e.returncode != 0:
    print("Erro ao copiar do Drive:")
    print(e.stderr)
else:
    print("Arquivos copiados para", tmp_dir)

# %% [code]
# 3) Configurar MinIO
minio_client = Minio(
    "minio:9000",
    access_key=os.getenv("MINIO_ROOT_USER", "admin"),
    secret_key=os.getenv("MINIO_ROOT_PASSWORD", "senhasegura"),
    secure=False
)
if not minio_client.bucket_exists('raw-data'):
    minio_client.make_bucket('raw-data')
    print("Bucket raw-data criado.")

# %% [code]
# 4) Upload de cada arquivo ao bucket raw-data/projeto-mba/
for root, dirs, files in os.walk(tmp_dir):
    for file in files:
        full = os.path.join(root, file)
        rel = os.path.relpath(full, tmp_dir)
        obj = f"projeto-mba/{rel}"
        minio_client.fput_object('raw-data', obj, full)
        print(f"Uploaded: {obj}")

# %% [markdown]
"""
**Conclusão**: O conteúdo de "PROJETO MBA" foi enviado ao MinIO em `raw-data/projeto-mba`, pronto para ingestão.
"""


Diretório temporário criado: /tmp/projeto_mba_9mm2erb4
Arquivos copiados para /tmp/projeto_mba_9mm2erb4
Uploaded: projeto-mba/Cópia de RE_ DADOS DE QUALIDADE DE MATÉRIA PRIMA1.zip
Uploaded: projeto-mba/Cópia de Cópia de Dados WIlson início 2023 1(in).csv
Uploaded: projeto-mba/Cópia de Dados Wilson início 2024 1(in).csv
Uploaded: projeto-mba/Realizando análise do repositório.pdf
Uploaded: projeto-mba/Política de Curadoria de Dados.pdf
Uploaded: projeto-mba/Status em 160425_15h53min.docx
Uploaded: projeto-mba/Cópia de Dados WIlson início 2023 1(in).csv
Uploaded: projeto-mba/Política de Curadoria de Dados.docx
Uploaded: projeto-mba/Cópia de Dados Wilson metade final 2024 1(in).csv
Uploaded: projeto-mba/unnamed.png
Uploaded: projeto-mba/CONTEXTOS E DOCUMENTOS DE INFORMAÇÃO PARA LLM/Resumo do Ambiente Operacional e Objetivos para o Próximo Chat.pdf
Uploaded: projeto-mba/CONTEXTOS E DOCUMENTOS DE INFORMAÇÃO PARA LLM/INVENTÁRIO DO QUE FOI FEITO EM 12_04.docx
Uploaded: projeto-mba/CONTEXTOS E 

'\n**Conclusão**: O conteúdo de "PROJETO MBA" foi enviado ao MinIO em `raw-data/projeto-mba`, pronto para ingestão.\n'